In [1]:
#evolutionary game
import pygame
import random
import numpy as np
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

pygame 2.0.1 (SDL 2.0.14, Python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# constants
inter_radius = 1
offspring_radius = 1
death_rate = 0.1

# colors
col_about_to_die = (200, 200, 225)
col_type1 = (255, 255, 0)
col_type2 = (255, 0, 255)
col_background = (10, 10, 40)
col_grid = (30, 30, 60)

In [3]:
# helper functions
def radius_rand(radius, r, c, cur, value):
    chosen_pos = None
    chosen_value = None
    R = np.arange(0, cur.shape[0], 1, dtype=int)
    C = np.arange(0, cur.shape[1], 1, dtype=int)
    
    # defining the begin and end of the neighberhood
    beginR = r-radius
    endR = r+radius 
    beginC = c-radius
    endC = c+radius

    for i in range(8*radius):
        rand_r = random.randint(beginR, endR)
        rand_c = random.randint(beginC, endC)
        
        rand_r = R[rand_r] if rand_r < cur.shape[0] else R[rand_r - cur.shape[0]]
        rand_c = C[rand_c] if rand_c < cur.shape[1] else R[rand_c - cur.shape[1]]
        
        if value == 0:
            if cur[rand_r, rand_c] == 0:
                chosen_pos = [rand_r, rand_c]
                chosen_value = 0
                return chosen_pos, chosen_value
        else: 
            if cur[rand_r, rand_c] != 0:
                chosen_pos = [rand_r, rand_c]
                chosen_value = cur[rand_r, rand_c]
                return chosen_pos, chosen_value
            
    return chosen_pos, chosen_value

def cemetery(cur):
    nxt = np.zeros((cur.shape[0], cur.shape[1]))
    for r, c in np.ndindex(cur.shape):
        nxt[r,c] = cur[r,c]
        
        death = np.random.uniform(0.0, 1.0, 1)
        
        if death <= death_rate:
            nxt[r,c]= 0 
    return nxt

In [4]:
# EA
def mating_pool(cur, fitMat):
    nxt = np.zeros((cur.shape[0], cur.shape[1]), dtype=np.int8)
    for r, c in np.ndindex(cur.shape):
        if nxt[r,c] == 0:
            vocal = cur[r,c]
            nxt[r,c] = vocal
            if vocal != 0:
                # lets kill some of the old generation
                death = np.random.uniform(0.0, 1.0, 1)

                if death <= death_rate:
                    nxt[r,c]= 0

                # pick a random individual from inetarction_radius to mate
                mate_pos, mate_value = radius_rand(inter_radius, r, c, cur, 1)
                if vocal >2 or vocal<0:
                    raise Exception("The type of the player is unknown")

                # Placing the offspring in offspring_radius according to A = [[1,2],[2,1]]
                if mate_value is not None:

                    nr_offspring = fitMat[int(vocal-1),int(mate_value-1)]
                    for i in range(nr_offspring):
                        offspring_pos, offspring_value = radius_rand(offspring_radius, r, c, cur, 0)
                        if offspring_value is not None:
                            nxt[offspring_pos[0], offspring_pos[1]] = vocal
    return nxt

In [5]:
def update(surface, cur, sz, fitMat):

    nxt = mating_pool(cur, fitMat)
     
    for r, c in np.ndindex(nxt.shape):
        if nxt[r, c] == 1:
            col = col_type1
        elif nxt[r, c] == 2:
            col = col_type2
        else:
            col = col_background
            
        pygame.draw.rect(surface, col, (c*sz, r*sz, sz-1, sz-1))
        
    return nxt

def init(dimx, dimy, pattern, pos):
    
    cells = np.zeros((dimy, dimx), dtype=np.int8)
    cells[pos[0]:pos[0]+pattern.shape[0], pos[1]:pos[1]+pattern.shape[1]] = pattern
    
    return cells

In [6]:
# main
def main(numGen, iniFram, pattern, pos, fitMat):
    cellsize, dimx, dimy = 6, 100, 100
    finalMatrix = None
    pygame.init()
    clock = pygame.time.Clock()
    surface = pygame.display.set_mode((dimx * cellsize, dimy * cellsize))
    pygame.display.set_caption("Evolutionary Game")
    generation = 1
    cells = init(dimx, dimy, pattern, pos)
    while True:
        dt = clock.tick(300)
        
        if generation >= iniFram:
            finalMatrix = np.dstack((finalMatrix, cells)) if finalMatrix is not None else cells
            
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return finalMatrix
            
        generation = generation+1
        surface.fill(col_grid)
        cells = update(surface, cells, cellsize, fitMat)
        pygame.display.update()
        
        if generation == numGen:
            pygame.quit()
            return finalMatrix

In [7]:
# call main
if __name__ == '__main__':
    inputs = []
    inputs.append([
        ['Generation_num', 40],
        ['iniFrames', 10],
        ['pattern' , np.array([ [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0],
                                [1,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [2,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]], dtype=np.int8)],
        ['Position', (5,10)],
        ['Fitness Matrix', np.array([ [1,2],
                                      [2,1]])],
        ])
    
    inputs.append([
        ['Generation_num', 60],
        ['iniFrames', 15],
        ['pattern' , np.array([ [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0],
                                [1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2,0,2,0,0,0,0,1,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
                                [0,0,1,2,2,1,2,2,1,2,2,1,1,1,2,2,2,2,2,0,0,0,0,0,1,1,1,1,2,1,1,1,1,1,1,0,0,0,0],
                                [0,0,1,2,2,1,2,2,1,2,2,1,1,1,2,2,2,2,2,0,0,0,0,0,1,1,1,1,2,1,1,1,1,1,1,0,0,0,0]], dtype=np.int8)],
        ['Position', (40,0)],
        ['Fitness Matrix', np.array([ [1,2],
                                      [2,1]])],
        ])
    
    finalMatrix = []
    for input_ in inputs:
        
        numGen, iniFram, pattern, pos, fitMat = input_[0][1], input_[1][1], input_[2][1], input_[3][1], input_[4][1]
        
        finalMatrix.append([input_, main(numGen, iniFram, pattern, pos, fitMat)])

In [8]:
np.savez_compressed('output_evo_game', evoGame=finalMatrix )
loaded = np.load('output_evo_game.npz',allow_pickle=True)
loaded['evoGame']

array([[list([['Generation_num', 40], ['iniFrames', 10], ['pattern', array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0],
       [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
        2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
        0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [9]:
# print('Input: \n', finalMatrix[0][0])
# print('a\nOutput shape: \n',finalMatrix[0][1].shape)


# print('Input: \n', finalMatrix[1][0])
# print('\nOutput shape: \n',finalMatrix[1][1].shape)